In [1]:
from distributed import Client
import os
#import logging
client = Client(silence_logs=50)

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 224,Total memory: 503.61 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37459,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 224
Started: Just now,Total memory: 503.61 GiB
Comm: tcp://127.0.0.1:41239,Total threads: 14
Dashboard: http://127.0.0.1:33061/status,Memory: 31.48 GiB
Nanny: tcp://127.0.0.1:33323,


In [3]:
os.environ["MODIN_ENGINE"] = "dask"

In [4]:
import modin.pandas as pd
import re
import string

In [5]:
BASE_DIR = '/home/u132668/'

In [36]:
train = pd.read_csv(os.path.join(BASE_DIR, 'data/train.csv'))

In [40]:
train.head()

,Story,Question,span_start,span_end,span_text,Answer
0,CHAPTER IV. \n\nNotwithstanding the earnest in...,What emotions did she show?,199.0,288.0,The countenance of this young lady exhibited a...,mirth and sadness
1,"(CNN) -- Polk County, Florida, detectives arre...",What do police believe the motive to be?,428.0,465.0,We believe that his motive is robbery,robbery
2,"Malawi (, or ; or [maláwi]), officially the Re...",Is it a large country?,653.0,702.0,Malawi is among the smallest countries in Africa.,No
3,CHAPTER XXXII. \n\nMR. GILMORE'S SUCCESS. \n\n...,What reminded him of Bullhampton?,1438.0,1448.0,Everything,Everything
4,"Computer security, also known as cybersecurity...",Besides breaking or staling a computer what ot...,NaN,-1.0,unknown,unknown


In [41]:
train.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 66611 entries, 0 to 66610
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Story       66611 non-null  object 
 1   Question    66611 non-null  object 
 2   span_start  58286 non-null  float64
 3   span_end    58035 non-null  float64
 4   span_text   58236 non-null  object 
 5   Answer      66611 non-null  object 
dtypes: float64(2), object(4)
memory usage: 3.0+ MB


In [42]:
train['NormAnswer'] = train['Answer'].apply(lambda x: x.lower().strip().strip(string.punctuation))

In [43]:
train_tf = train[(train['NormAnswer'] == 'true') | (train['NormAnswer'] == 'false')].copy()
train_yn = train[(train['NormAnswer'] == 'yes') | (train['NormAnswer'] == 'no')].copy()

In [44]:
print('Number of true/false questions: ' + str(len(train_tf)))
print('Number of yes/no questions: ' + str(len(train_yn)))
print('Number of unknown answers: ' + str(len(train[train['NormAnswer'] == 'unknown'])))

Number of true/false questions: 61
Number of yes/no questions: 11210
Number of unknown answers: 772


In [12]:
train.drop(train_tf.index, inplace=True)
train.drop(train_yn.index, inplace=True)
train.drop(train[train['NormAnswer'] == 'unknown'].index, inplace=True)

In [13]:
def preprocess_text(text):
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub('--', ' ', text)
    text = re.sub('\s{2,}', ' ', text)
    text = text.strip()
    text = text.lower()
    return text

In [14]:
train['Story'] = train['Story'].apply(preprocess_text)

In [15]:
train['answer_start'] = train.apply(lambda row: row['Story'].rfind(row['NormAnswer']), axis=1)

In [16]:
train_story_contained_ans = train[train['answer_start']>=0]

In [17]:
print('Number of examples with answer present in context: ' + str(len(train_story_contained_ans)))

Number of examples with answer present in context: 44099


In [46]:
train_tf.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 61 entries, 631 to 64970
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Story       61 non-null     object 
 1   Question    61 non-null     object 
 2   span_start  49 non-null     float64
 3   span_end    54 non-null     float64
 4   span_text   56 non-null     object 
 5   Answer      61 non-null     object 
 6   NormAnswer  61 non-null     object 
dtypes: float64(2), object(5)
memory usage: 3.8+ KB


In [58]:
train_tf[train_tf['span_text'].isnull() == True]

,Story,Question,span_start,span_end,span_text,Answer,NormAnswer
12429,Linux is a name which broadly denotes a family...,True or False: Most Linux distributions are ju...,485.0,NaN,NaN,false,false
15127,(CNN)It was 3:30 in the morning when Robert Al...,"True or False: The first sign mentioned said ""...",NaN,1169.0,NaN,true,true
36297,CHAPTER V \n\nFOLLOWING A TRAIL \n\nFrank had ...,True or False: Faulkner had been riding a horse.,417.0,NaN,NaN,True,true
36601,Chapter IX \n\nNobble \n\n\n\nDuring the two d...,True or False: Caldigate thought a closer rela...,1269.0,1372.0,NaN,True.,true
56771,CHAPTER V. \n\nMOHUN APPEARS FOR THE LAST TIME...,True of False: General Webb was cowardly.,1499.0,1514.0,NaN,false,false


In [47]:
train_yn.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 11210 entries, 2 to 66603
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Story       11210 non-null  object 
 1   Question    11210 non-null  object 
 2   span_start  9820 non-null   float64
 3   span_end    9808 non-null   float64
 4   span_text   9823 non-null   object 
 5   Answer      11210 non-null  object 
 6   NormAnswer  11210 non-null  object 
dtypes: float64(2), object(5)
memory usage: 700.6+ KB


In [59]:
train_yn[train_yn['span_text'].isnull() == True]

,Story,Question,span_start,span_end,span_text,Answer,NormAnswer
33,"BOGOTA, Colombia (CNN) -- ""Easy money, fast an...",Substantially?,997.0,1018.0,NaN,yes,yes
36,Norfolk Island (i/ˈnɔːrfək ˈaɪlənd/; Norfuk: N...,Was it used the entire time in that manner?,778.0,870.0,NaN,No,no
47,CHAPTER IV \n\nTHE WAY INTO PRINT \n\nSam Cott...,were they the one who talked about the phone c...,872.0,1052.0,NaN,Yes,yes
70,New York—often called New York City or the Cit...,is food one of them?,409.0,507.0,NaN,No,no
92,"CHAPTER V \n\nGETTING ACQUAINTED \n\n""Dick, we...",Is he just a distant acquaintance?,1212.0,1242.0,NaN,no,no
...,...,...,...,...,...,...,...
66411,CHAPTER IX.—THE MAD ELEPHANT. \n\nFrom Middlet...,Did Leo fight Broxton?,1207.0,1287.0,NaN,yes,yes
66537,CHAPTER XI \n\nWITH DEJAH THORIS \n\nAs we rea...,Do males and females murder each other?,622.0,670.0,NaN,No,no
66573,One day a young boy went to visit a toy store....,Were they friends before they played cars in t...,385.0,466.0,NaN,no,no
66592,"CHAPTER XLI. SPEECHIFYING. \n\nOn the Monday, ...",Did people believe the affairs of the parrish?,595.0,673.0,NaN,yes,yes
